# What's new in Python 3.4 and below?

This show-cases new features in the form of code snippets, based on these documents:

* [What’s New In Python 3.0](https://docs.python.org/3.6/whatsnew/3.0.html)
* [What’s New In Python 3.1](https://docs.python.org/3.6/whatsnew/3.1.html)
* [What’s New In Python 3.2](https://docs.python.org/3.6/whatsnew/3.2.html)
* [What’s New In Python 3.3](https://docs.python.org/3.6/whatsnew/3.3.html)
* [What’s New In Python 3.4](https://docs.python.org/3.6/whatsnew/3.4.html)

I won't mention stuff most people know from Python 2.7 backports like print-as-a-function, integer division, unicode strings, and so on.

Python 3.4 introduces by-default certificate verification for HTTP clients (PEP 476) and makes `pip` a commodity.

## New and Changed Syntax

Raising exceptions can now include a root cause (`raise ‹exc› from ‹cause›`), and catching is done with `except ‹exc› as ‹cause›`.

Dictionary comprehensions:

In [1]:
{x: x**2 for x in range(5)}

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

Set literals and set comprehensions:

In [2]:
print({1, 2, 1}, set(), type({}), {x.upper() for x in __name__}, sep='\n')

{1, 2}
set()
<class 'dict'>
{'_', 'I', 'A', 'N', 'M'}


New octal and binary literals (the old octal literals are gone).

In [3]:
0o720, oct(464), 0b1010, bin(10)

(464, '0o720', 10, '0b1010')

Bytes literals and `bytes()`:

In [4]:
b'abc\234', bytes('ab¢', encoding='utf-8')

(b'abc\x9c', b'ab\xc2\xa2')

## New Library Modules

-   **asyncio**: New provisional API for asynchronous I/O
-   ensurepip: Bootstrapping the pip installer
-   **enum**: Support for enumeration types
-   **pathlib**: Object-oriented filesystem paths
-   selectors: High-level and efficient I/O multiplexing, built upon the select module primitives
-   **statistics**: A basic numerically stable statistics library
-   tracemalloc: Trace Python memory allocations

## Changes to Built-ins

### Ordered Dictionaries (v3.1, PEP 372)

The new `collections.OrderedDict` class allows you to retain insertion order of items. Note that Python 3.7 makes this a language feature for `dict`, and 3.6 already implements it *incidentally*.

In [5]:
from collections import OrderedDict
o = OrderedDict(z=1, x=2, a=3)
o, dict(o)

(OrderedDict([('z', 1), ('x', 2), ('a', 3)]), {'z': 1, 'x': 2, 'a': 3})

If a new entry overwrites an existing entry, the original insertion position is left unchanged.

In [6]:
o['x'] = -2
o

OrderedDict([('z', 1), ('x', -2), ('a', 3)])

Deleting an entry and reinserting it will move it to the end.

In [7]:
del o['x']; o['x'] = 4
o

OrderedDict([('z', 1), ('a', 3), ('x', 4)])

The standard library takes advantage of ordering at several places.

### Format Specifier for Thousands Separator (v3.1, PEP 378)

The built-in `format` function and `str.format` offer a simple, non-locale aware way to format a number with a thousands separator. Locale-aware applications should use the existing `n` format specifier which already has some support for thousands separators.

In [8]:
format(1234567, ',d')

'1,234,567'

In [9]:
format(1234567.89, ',.2f')

'1,234,567.89'

In [10]:
format(12345.6 + 8901234.12j, ',f')

'12,345.600000+8,901,234.120000j'

In [11]:
import decimal
format(decimal.Decimal('1234567.89'), ',f')

'1,234,567.89'

The supported types are `int`, `float`, `complex`, and `decimal.Decimal`.